# NASA Exoplanet Archive PLATON Atmospheric Retrieval
This notebook utilizes the PLanetary Atmospheric Transmission for Observer Noobs (PLATON) tool ([Zhang, M. et al., 2019](https://ui.adsabs.harvard.edu/abs/2019PASP..131c4501Z/abstract), [Zhang, M. et al., 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...899...27Z/abstract), [Source on Github](https://github.com/ideasrule/platon), [Documentation](https://platon.readthedocs.io)) for use with [Transmission Spectroscopy data](https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=transitspec) from the Exoplanet Archive.

This notebook is meant to require minimal user input, unless you want to provide your own data. Each step will either begin with *(Play)* or **(Input)**. No editing is necessary for the *(Play)* steps, just hover over the cell and press the "Play" button on the left. The **(Input)** steps will tell you what can be changed.

**NOTE:** This has not been tested on all planets with transmission spectroscopy data on the Exoplanet Archive, so your results may vary!

**Estimated Runtime**: 1 hour

*Author*: [Kevin Hardegree-Ullman](http://kevinkhu.com), modified by [Sam Grunblatt](skgrunblatt.github.io)

*Last Modified*: November 5, 2024

1.   *(Play)* Install some dependencies and the PLATON package.



In [ ]:
!apt-get install libopenblas-dev
!pip install git+https://github.com/ideasrule/platon.git

2.   *(Play)* Import some packages!



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import io
import requests
from platon.plotter import Plotter
from platon.fit_info import FitInfo
from platon.combined_retriever import CombinedRetriever
from platon.constants import R_sun, R_jup, M_jup, R_earth, M_earth
print('Packages successfully imported!')

3.   *(Play)* Read in the [transmission spectroscopy table](https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=transitspec) from the Exoplanet Archive and list the available planets.



In [ ]:
url="https://raw.githubusercontent.com/skgrunblatt/exoplanet_atmospheres/refs/heads/main/transitspec_2024.11.05_13.31.29.csv?dl=1"
tstable=pd.read_csv(url, comment='#', header=0)
tstable.dropna(subset=['plnratror','bandwidth','centralwavelng'], inplace=True, axis=0)
tstable.sort_values('plntname',inplace=True)
tstable.reset_index(drop=True,inplace=True)

print('Here are all the available planets: ',tstable.plntname.unique())

4. **(Input)** Select your favorite planet from the list above.

In [ ]:
planet = 'WASP-121 b' # Edit the name within the quotes. Make sure the exact string matches a planet in the list above.

if tstable['plntname'].str.contains(planet).any():
  print("You have selected planet "+planet+'.')
else:
  print("Please check your planet name, it doesn't appear to be in the list above.")

5. *(Play)* Plot the data! Check that things look okay.

In [ ]:
# Extract data from the table above for only the selected planet.
df = tstable.loc[(tstable['plntname'] == planet) & (tstable['centralwavelng'] > 0.3)] #This truncates the data to >0.3 microns, the limit of the models used below.
df = df.dropna(subset=['bandwidth', 'plnratror', 'plnratrorerr1', 'plnratrorerr2'])
df = df.sort_values(by='centralwavelng')
df = df.reset_index(drop=True)

# Make the plot
plt.figure(figsize=(10,8))
plt.title(planet+' Transit Data', fontsize=26)
plt.errorbar(x=df.centralwavelng,y=df.plnratror,xerr=df.bandwidth,yerr=[df.plnratrorerr1,-df.plnratrorerr2],fmt='o',color='red')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel('Wavelength ($\mathrm{\mu}$m)',fontsize=24)
plt.ylabel('$R_p/R_{\star}$',fontsize=24)

# plt.xscale('log') #uncomment if necessary

6. *(Play)* Download system parameter data from the Exoplanet Archive.

In [ ]:
p2ascii = planet.replace(" ","%20")

url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync?query=select+*+from+ps+where+pl_name+=+'"+p2ascii+"'+and+default_flag+=+1+&format=csv"

plsyspars=pd.read_csv(url, comment='#', header=0)
plsyspars

7. *(Play)* Take the basic planet and stellar parameters from the table above. Feel free to **(Input)** your own values if you have them.


In [ ]:
#Reference quantities
R_earth = 6371009 #meters
R_jup = 69950000 #meters
R_sun = 695700000 #meters
M_earth = 5.97e24 #kilograms
M_jup = 1.898e27 #kilograms
M_sun = 1.988435e30 #kilograms
G = 6.6743015e-11 #m^3 kg^-1 s^-2

#Planet Parameters
R_guess = plsyspars['pl_rade'][0] * R_earth #Planet radius guess in Earth radii, if that wasn't obvious already
R_err = np.mean([np.abs(plsyspars['pl_radeerr1'][0]),np.abs(plsyspars['pl_radeerr2'][0])]) * R_earth #Planet radius error in Earth radii
print(planet+' Planet Parameters:')
print('R_planet = '+str(round(R_guess/R_earth,2))+' \u00B1 '+str(round(R_err/R_earth,2))+' R_Earth = '+str(round(R_guess/R_jup,2))+' \u00B1 '+str(round(R_err/R_jup,2))+' R_Jupiter')

M_pl = plsyspars['pl_bmasse'][0] * M_earth #Planet mass (or msini) in Earth masses
M_pl_err = np.mean([np.abs(plsyspars['pl_bmasseerr1'][0]),np.abs(plsyspars['pl_bmasseerr2'][0])]) * M_earth #Planet mass error in Earth masses
print('M_planet = '+str(round(M_pl/M_earth,2))+' \u00B1 '+str(round(M_pl_err/M_earth,2))+' M_Earth = '+str(round(M_pl/M_jup,2))+' \u00B1 '+str(round(M_pl_err/M_jup,2))+' M_Jupiter')

grav = G*M_pl/(R_guess**2)
print('Surface Gravity = '+str(round(grav,2))+' m/s^2')

T_guess = 2400#plsyspars['pl_eqt'][0] #Planet equilibrium temperature guess in Kelvin
print('T_eq = '+str(int(T_guess))+' K')
if T_guess < 300 or T_guess > 2600:
  print('*****STOP!***** T_eq is beyond the temperature range of the current model set. The following code will not work. Sorry :(')

#Stellar Host Parameters
Rs = plsyspars['st_rad'][0] * R_sun #Star radius in Sun radii
Rs_err = np.mean([np.abs(plsyspars['st_raderr1'][0]),np.abs(plsyspars['st_raderr2'][0])]) * R_sun #Star radius measurement error in Sun radii
print('\nStellar Host Parameters')
print('R_star = '+str(round(Rs/R_sun,2))+' \u00B1 '+str(round(Rs_err/R_sun,2))+' R_Sun')

T_star = 6400#plsyspars['st_teff'][0] #Star effective temperature in Kelvin
print('T_eff = '+str(int(T_star))+' K')

8. *(Play)* If you have values for planetary atmospheric metallicity (log Z) or the planetary C/O ratio, **(Input)** them below, otherwise leave them at the default values.

In [ ]:
logZ_guess = 1.09 #Planetary atmospheric metallicity relative to the Sun. If you don't know this, set to 0.
CO_guess = 0.49 #Planetary C/O ratio. If you don't know this, set to 0.54 which is the solar value.
print('log(Z) = '+str(logZ_guess))
print('C/O = '+str(CO_guess))

9. *(Play)* *Run the PLATON retrieval!* This will run 100 emcee steps and take ~**35 to 75 minutes** (typically depending on the amount of input data, on first run a 10GB file is also downloaded), so feel free to work on something else for about an hour... or if you're at home do yourself a favor and go bake some cookies! By the time the cookies are done, your code will have finished (probably), your place will smell like cookies, and, best of all, you'll have some cookies! After all 100 steps have run, the best fit results will print below and also save to a file *BestFit.txt* in the Files panel to the left.

In [ ]:
#Convert the data input to arrays
def winputs():
  wave_bins = []
  depths = []
  errors = []
  for i in range(len(df)):
    wave_bins.append([df['centralwavelng'][i]-df['bandwidth'][i]/2,df['centralwavelng'][i]+df['bandwidth'][i]/2])
    depths.append((df['plnratror'][i])**2)
    errors.append((df['plnratrorerr1'][i])**2)
  return 1e-6*np.array(wave_bins), np.array(depths), np.array(errors)

wbins, wdepths, werrors = winputs()
bins=wbins
depths=wdepths
errors=werrors

#create a Retriever object
retriever = CombinedRetriever()

#create a FitInfo object and set best guess parameters
fit_info = retriever.get_default_fit_info(
    Rs=Rs, Mp=M_pl, Rp=R_guess, T=T_guess,
    logZ=logZ_guess, CO_ratio=CO_guess, log_cloudtop_P=3,
    log_scatt_factor=0, scatt_slope=4, error_multiple=1, T_star=T_star)

#Add fitting parameters - this specifies which parameters you want to fit
#e.g. since we have not included cloudtop_P, it will be fixed at the value specified in the constructor

fit_info.add_gaussian_fit_param('Rs', Rs_err)
fit_info.add_gaussian_fit_param('Mp', M_pl_err)

# Here, emcee is initialized with walkers where R is between 0.9*R_guess and
# 1.1*R_guess.  However, the hard limit on R is from 0 to infinity.
fit_info.add_uniform_fit_param('Rp', 0, np.inf, 0.9*R_guess, 1.1*R_guess)

fit_info.add_uniform_fit_param('T', 1000, 3000, 0.5*T_guess, 1.5*T_guess)
fit_info.add_uniform_fit_param("log_scatt_factor", 0, 5, 0, 1)
fit_info.add_uniform_fit_param("logZ", -1, 3)
fit_info.add_uniform_fit_param("log_cloudtop_P", -0.99, 5)
fit_info.add_uniform_fit_param("error_multiple", 0, np.inf, 0.5, 5)

#Use Nested Sampling to do the fitting
result = retriever.run_emcee(bins, depths, errors,
                             None, None, None,
                             fit_info, nwalkers=50, nsteps=100,
                             rad_method="xsec" #"ktables" for corr-k
)

with open("example_retrieval_result.pkl", "wb") as f:
    pickle.dump(result, f)

#Useful members: result.chain, result.lnprobability, result.flatchain, result.flatlnprobability

10. *(Play)* Plot your data and best fit retrieved spectrum. This will also save a file called *planet name*_best_fit.png in the Files panel to the left.


In [ ]:
plotter = Plotter()
plt.figure(figsize=(10,8))
plotter.plot_retrieval_transit_spectrum(result, prefix="best_fit")
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel('Wavelength (microns)',fontsize=24)
plt.ylabel('Transit depth',fontsize=24)
plt.xscale('linear') #uncomment as needed
plt.legend(fontsize='18')

11. *(Play)* Plot the 2D posteriors from the atmospheric retrieval. This will also save a file called *planet name*_emcee_corner.png in the sample_data folder to the left. If your posteriors do not look great, you might want to tune the parameters in Steps 7 and 8.

In [ ]:
plotter.plot_retrieval_corner(result)

You have reached the end of this tutorial, congratulations!

#Development Roadmap

*   Edit for easy user upload of their own data.
*   Edit to allow/convert different transit depth parameters (e.g., Rp/Rs, (Rp/Rs)^2, %, mmag).
*   Acquire transit data from community repositories (e.g., [Exoplanet Watch](https://exoplanets.nasa.gov/exoplanet-watch/about-exoplanet-watch/overview/)).
*   Color transit data by input source to allow checking for inconsistent data or removing sources.
*   Allow for combining/averaging measurements at the same wavelength/bandpass.